## Coppersmith attack

In [1]:
def format_large_number(num):
    abs_num = abs(num)
    
    if abs_num < 1e6:
        return "{:.2f}".format(num)
    elif abs_num < 1e15:
        # Convert to float before formatting
        return "{:.2e}".format(num.n())
    else:
        # For very large numbers, show the order of magnitude
        if abs_num == 0:
            return "0.00e+00"
        order = int(log(abs_num, 10))
        return "{:.2e} * 10^{}".format(num.n() / 10**order, order)

# display matrix picture with 0 and the approximated value of the matrix elements
def print_matrix(basis_matrix, bound):
    for i in range(basis_matrix.dimensions()[0]):
        a = ('%02d ' % i)
        for j in range(basis_matrix.dimensions()[1]):
            if j == i:
                a += '0' if basis_matrix[i,j] == 0 else format_large_number(basis_matrix[i, j])
            if j == basis_matrix.dimensions()[1] - 1:
                a += ' '
            else:
                a += ' '
        if basis_matrix[i, j] >= bound:
            a += '~'
        print(a)

In [2]:
def coppersmith(pol, N, m, X):
    poly_degree = pol.degree()
    # m_dim of lattice is dimension of matrix
    m_dim = poly_degree*m

    if not pol.is_monic():
        raise ArithmeticError("Polynomial must be monic.")

    w = m_dim
    # determinant
    detL = RR(N^(poly_degree) * X^(m * (m + 1) / 2))
    # Check the Coppersmith condition
    lhs = RR(2^(w/4) * detL^(1/w))
    rhs = RR(N^m / sqrt(w))

    if lhs < rhs:
        print("The condition satisfied.\n")
    else:
        print("The condition not satisfied.\n")


    # change ring of pol and x
    polZ = pol.change_ring(ZZ)
    x = polZ.parent().gen()

    # compute polynomials 
    polynomials_g = []
    for i in range(m):
        for j in range(poly_degree):
            polynomials_g.append((x * X)**j * N**(m - i) * polZ(x * X)**i)
    print("Polynomial: ", polynomials_g)

    # construct lattice matrix
    matrix = Matrix(ZZ, m_dim)

    for i in range(m_dim):
        for j in range(i+1):
            matrix[i, j] = polynomials_g[i][j]

    # display basis matrix
    print("Only diagonal elements are shown since they form the determinant:")
    print_matrix(matrix, N^m)

    # apply LLL algorithm
    matrix = matrix.LLL()

    # Transform shortest vector in polynomial    
    new_polynomial = 0
    for i in range(m_dim):
        new_polynomial += x**i * matrix[0, i] / X**i

    # Factor polynomial
    possible_roots = new_polynomial.roots()
    print("Possible roots:", possible_roots)

    # test roots
    filtered_roots = []
    for root in possible_roots:
        if root[0].is_integer():
            result = polZ(ZZ(root[0]))
            if gcd(N, result) >= N^(1/m):
                filtered_roots.append(ZZ(root[0]))
                
    # Print intermediate values
    print("\n# Intermediate Values")
    print("Possible roots:", possible_roots)
    print("Filtered roots:", filtered_roots)

    return filtered_roots

### Example 1: Large known bits of Ciphertext

In [3]:
# RSA generator
k_bits = 1024
p = next_prime(2^int(round(k_bits/2)))
q = next_prime( round(pi.n()*p) )
N = p*q

# p_new = known p bits + unknown bits
hidden = 200
unk_bits = ZZ.random_element(0, 2^hidden-1)
p_new = p + unk_bits 

F.<x> = PolynomialRing(Zmod(N)) 
pol = x - p_new
poly_degree = pol.degree()

# set arbitrary
epsilon = 1
# set poly degree
m = 3
# upper bound
X = ceil(N**((1/poly_degree) - epsilon))

valid_roots = coppersmith(pol, N, m, X)


print("Value of unknown bits:", p_new - p)
print("Possible roots:", valid_roots)
print()
# Check if the roots correspond to p or q
print("Original p: ", p, "\nOriginal q: ", q)
print("Roots found:")
for root in valid_roots:
    print("root: ", root)
    if N % root == 0:
        if root == p:
            print("calculated p: ", root)
        elif root == q:
            print("calculated q: ", root)
        else:
            if p % root == 0:
                print("p factors are: ", root, "and", p/root)
            elif q % root == 0:
                print("q factors are: ", root, "and", q/root)

The condition satisfied.

Polynomial:  [180134250895969534680170752961072129141412273843745708988621443697691128816768197377296197502043191540968858501401767209021252515558499237673104175663474562118198395050224182448694887992931007367538784441358187642660782552558325602999224430122482679733777275228550958957373383842670996222350100807493581849593270006501731563165996288439262000275515683319057637958154155827213267791036456492438159997772343848396069547496958147658891337029440034928392174431779192152603429122980737558483426218912167054044514087265552413628174912034640062373372419747689370913530597940533923344779569246597206951463752117659618031967367099779213596303123639744994304531607005241231374328228377848555592149606108988348904413617623239773673430402468852635161516176750387628456408737634165562983404880415609191700540306360128682688866840470251167353698043275988981035255707388419770811751464876516968315411922396433377981151014156076003949952927211, 3189560653514426156553658938986

### Example 2: Stereotyped message with small e in RSA

In [6]:
n = random_prime(2^160)*random_prime(2^160)
m = Integer('ourlastcryptographyassignment',36)
c = m^3 % n

a = Integer('ourlastcryptography0000000000',36)

X = Integer('zzzzzzzzzz',36)

M = matrix([[X^3, 3*X^2*a, 3*X*a^2, a^3-c],
           [0, n*X^2, 0, 0],
           [0, 0, n*X, 0],
           [0, 0, 0, n]])
B = M.LLL()
Q = B[0][0]*x^3/X^3 + B[0][1]*x^2/X^2 + B[0][2]*x/X + B[0][3]

roots = Q.roots(ring=ZZ)[0][0].str(base=36)
print("Unknown part of message:", roots)

Unknown part of message: assignment
